In [ ]:
import sys
import os
import torch
from src.models import FashionMNISTModel
from datasets import IncrementalFashionMNIST
from src.eval import evaluate
from src.utils import add_heads


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
datamanager = IncrementalFashionMNIST()

model_1 = FashionMNISTModel(output_size=2)
model_1.to(device)
model_1.load_state_dict(torch.load("experiments/fashion_mnist_incremental/model_after_task_1.pth"))
model_2 = FashionMNISTModel(output_size=4)
model_2.to(device)
model_2.load_state_dict(torch.load("experiments/fashion_mnist_incremental/model_after_task_2.pth"))
add_heads(model_1, 8)  
add_heads(model_2, 6)




## Compute pearson correlation between outputs of different models (normal training)
def compute_pearson(model_a, model_b, dataloader, device):
    model_a.eval()
    model_b.eval()
    correlations = []

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs_a = model_a(inputs).cpu()  # 确保在 CPU
            outputs_b = model_b(inputs).cpu()

            for out_a, out_b in zip(outputs_a, outputs_b):
                # Flatten in case outputs are multidimensional
                out_a_flat = out_a.flatten()
                out_b_flat = out_b.flatten()

                # 标准化
                out_a_std = (out_a_flat - out_a_flat.mean()) / out_a_flat.std()
                out_b_std = (out_b_flat - out_b_flat.mean()) / out_b_flat.std()

                corr = torch.dot(out_a_std, out_b_std) / (out_a_flat.numel() - 1)
                correlations.append(corr.item())

    return sum(correlations) / len(correlations)  #  Pearson correlation

data_loader = datamanager.get_loader(mode='test', label=range(10))
pearson_corr = compute_pearson(model_1, model_2, data_loader, device)
print("Average Pearson correlation between model 1 and model 2:", pearson_corr)

Average Pearson correlation between model 1 and model 2: 0.16533979436223867
